In [2]:
#!git clone https://github.com/IndoNLP/indonlu.git

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.2 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 6.6 MB 63.2 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import os, sys
sys.path.append('../')
os.chdir('../')

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer

from content.drive.MyDrive.TA.indonlu.utils.forward_fn import forward_sequence_classification
from content.drive.MyDrive.TA.indonlu.utils.metrics import hadits_classification_metrics_fn
from content.drive.MyDrive.TA.indonlu.utils.data_utils import HaditsClassificationDataset, HaditsClassificationDataLoader


In [5]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [6]:
# Set random seed
set_seed(26092020)

In [7]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = HaditsClassificationDataset.NUM_LABELS

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# model.summary()

In [9]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [10]:
count_param(model)

124500557

In [11]:
train_dataset_path = '/content/drive/MyDrive/TA/bert_train/nr/bukhari_train.csv'
valid_dataset_path = '/content/drive/MyDrive/TA/bert_train/nr/bukhari_valid.csv'
test_dataset_path = '/content/drive/MyDrive/TA/bert_train/nr/bukhari_test.csv'

In [12]:
train_dataset = HaditsClassificationDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = HaditsClassificationDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = HaditsClassificationDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = HaditsClassificationDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=8, num_workers=4, shuffle=True)  
valid_loader = HaditsClassificationDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=8, num_workers=4, shuffle=False)  
test_loader = HaditsClassificationDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=8, num_workers=4, shuffle=False)

In [13]:
w2i, i2w = HaditsClassificationDataset.LABEL2INDEX, HaditsClassificationDataset.INDEX2LABEL
print(w2i)
print(i2w)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76}
{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 

In [14]:
# ___________________________________________-

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fine Tuning & Evaluation

In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
optimizer = optim.Adam(model.parameters(), lr=5e-6)
model = model.cuda()

In [18]:
!torch.cuda.empty_cache()
!nvidia-smi

/bin/bash: -c: line 1: syntax error: unexpected end of file
Sun Aug  7 18:49:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W /  70W |   1138MiB / 15109MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [19]:
torch.no_grad()

In [20]:
# Train
loss_list = []
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
n_epochs = 20
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = hadits_classification_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = hadits_classification_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = hadits_classification_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))
    loss_list.append(total_loss/(i+1))
    accuracy_list.append(metrics['ACC'])
    precision_list.append(metrics['PRE'])
    recall_list.append(metrics['REC'])
    f1_list.append(metrics['F1'])

(Epoch 1) TRAIN LOSS:4.0641 LR:0.00000500: 100%|██████████| 701/701 [04:37<00:00,  2.53it/s]


(Epoch 1) TRAIN LOSS:4.0641 ACC:0.13 F1:0.11 REC:0.13 PRE:0.15 LR:0.00000500


  0%|          | 0/88 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
VALID LOSS:3.9370 ACC:0.00 F1:0.00 REC:0.00 PRE:0.00:   1%|          | 1/88 [00:00<00:19,  4.55it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_star

(Epoch 1) VALID LOSS:3.3806 ACC:0.34 F1:0.28 REC:0.34 PRE:0.33


(Epoch 2) TRAIN LOSS:3.0065 LR:0.00000500: 100%|██████████| 701/701 [04:48<00:00,  2.43it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

(Epoch 19) VALID LOSS:0.9537 ACC:0.80 F1:0.78 REC:0.80 PRE:0.79


(Epoch 20) TRAIN LOSS:0.0198 LR:0.00000500: 100%|██████████| 701/701 [04:50<00:00,  2.42it/s]


(Epoch 20) TRAIN LOSS:0.0198 ACC:1.00 F1:1.00 REC:1.00 PRE:1.00 LR:0.00000500


  0%|          | 0/88 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
VALID LOSS:1.4363 ACC:0.75 F1:0.63 REC:0.61 PRE:0.67:   1%|          | 1/88 [00:00<00:19,  4.55it/s]/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_star

(Epoch 20) VALID LOSS:0.9926 ACC:0.80 F1:0.78 REC:0.79 PRE:0.78


In [21]:
report = pd.DataFrame({'acc': accuracy_list, 'precision': precision_list, 'recall':recall_list, 'f1_score':f1_list, 'loss': loss_list}, columns=['acc', 'precision', 'recall','f1_score', 'loss'])

In [22]:
output = '/content/drive/MyDrive/TA/bert_result/metrics_nrr.csv'
report.to_csv(output, index=False)

In [24]:
# Evaluate on test
# model.eval()
# torch.set_grad_enabled(False)

# total_loss, total_correct, total_labels = 0, 0, 0
# list_hyp, list_label = [], []

# pbar = tqdm(test_loader, leave=True, total=len(test_loader))
# for i, batch_data in enumerate(pbar):
#     _, batch_hyp, _ = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
#     list_hyp += batch_hyp

# # Save prediction
# df = pd.DataFrame({'label':list_hyp}).reset_index()
# df.to_csv('pred.txt', index=False)

# print(df)

In [ ]:
text = ' wahyu'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text:  | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

In [ ]:
text = "apa"
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text:  | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

In [ ]:
import scipy.sparse as sparse

def predict_hadits(hadits):
  result = []
  for i in tqdm(range(0, len(hadits))):
    text = hadits['indo'][i]
    subwords = tokenizer.encode(text)
    subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

    # Mengurangi panjang words, jika lebih dari 512
    arr = subwords[0]    
    if len(arr) > 512:
      arr = arr[:512]
      arr = np.reshape(arr,(1,512))
      subwords = arr
    logits = model(subwords)[0]
    label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    result.append(label)
  return result

def reading_data_result(data_file):
    data_file = pd.read_json(data_file)
    data_file = data_file.sort_values(by=['haditsId'])
    data_file = data_file.reset_index()
    data_file = data_file[["haditsId" , "kitabId","indo", "arab" ]]
    return data_file


In [ ]:
hadits_data_loc = [
    '/content/drive/MyDrive/TA/json_data/abudaud.json',
    '/content/drive/MyDrive/TA/json_data/darimi.json',
    '/content/drive/MyDrive/TA/json_data/ibnumajah.json',
    '/content/drive/MyDrive/TA/json_data/malik.json',
    '/content/drive/MyDrive/TA/json_data/muslim.json',
    '/content/drive/MyDrive/TA/json_data/nasai.json',
    '/content/drive/MyDrive/TA/json_data/tirmidzi.json'
]
hadits_ns_loc = [
    '/content/drive/MyDrive/TA/noSanad/abudaud.csv',
    '/content/drive/MyDrive/TA/noSanad/darimi.csv',
    '/content/drive/MyDrive/TA/noSanad/ibnumajah.csv',
    '/content/drive/MyDrive/TA/noSanad/malik.csv',
    '/content/drive/MyDrive/TA/noSanad/muslim.csv',
    '/content/drive/MyDrive/TA/noSanad/nasai.csv',
    '/content/drive/MyDrive/TA/noSanad/tirmidzi.csv'
]
hadits_nsn_loc = [
    '/content/drive/MyDrive/TA/noSanad_name/abudaud.csv',
    '/content/drive/MyDrive/TA/noSanad_name/darimi.csv',
    '/content/drive/MyDrive/TA/noSanad_name/ibnumajah.csv',
    '/content/drive/MyDrive/TA/noSanad_name/malik.csv',
    '/content/drive/MyDrive/TA/noSanad_name/muslim.csv',
    '/content/drive/MyDrive/TA/noSanad_name/nasai.csv',
    '/content/drive/MyDrive/TA/noSanad_name/tirmidzi.csv'
]
hadits_ns_save_loc = [
    '/content/drive/MyDrive/TA/bert_result/model4/ns/abudaud.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/ns/darimi.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/ns/ibnumajah.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/ns/malik.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/ns/muslim.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/ns/nasai.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/ns/tirmidzi.csv'
]
hadits_nsn_save_loc = [
    '/content/drive/MyDrive/TA/bert_result/model4/nr/abudaud.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/nr/darimi.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/nr/ibnumajah.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/nr/malik.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/nr/muslim.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/nr/nasai.csv',
    '/content/drive/MyDrive/TA/bert_result/model4/nr/tirmidzi.csv'
]

In [ ]:
for i in range(len(hadits_nsn_loc)):
  hadits = pd.read_csv(hadits_nsn_loc[i])
  hadits = predict_hadits(hadits)
  final = reading_data_result(hadits_data_loc[i])
  final['label'] = hadits
  file_output = hadits_nsn_save_loc[i]
  final.to_csv(file_output, index=False)

In [ ]:
hadits = pd.read_csv('/content/drive/MyDrive/TA/bert_result/model4/ns/muslim.csv')

In [ ]:
result = hadits.groupby('label').describe()
file_output = '/content/drive/MyDrive/TA/bert_result/model4/log_ns/muslim.csv'
result.to_csv(file_output, index=False)

In [ ]:
# hadits = pd.read_csv('/content/drive/MyDrive/TA/noSanad/tirmidzi.csv')
# hadits = predict_hadits(hadits)
# final = reading_data_result('/content/drive/MyDrive/TA/json_data/tirmidzi.json')
# final['label'] = hadits
# final.head()
# file_output = '/content/drive/MyDrive/TA/bert_result/model4/ns/tirmidzi.csv'
# final.to_csv(file_output, index=False)

In [ ]:
final.groupby('label').describe()

In [ ]:
model.save_pretrained('/content/drive/MyDrive/TA/model/4')

In [ ]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/TA/model/4')
model.bert.load_state_dict(model.bert.state_dict())